<a href="https://colab.research.google.com/github/yuugiouduele/AImodel/blob/main/Newmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import networkx as nx
from scipy import sparse, linalg
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Optional, NamedTuple
import warnings
from dataclasses import dataclass, field
from enum import Enum
import itertools
import random

warnings.filterwarnings('ignore')

# --- 既存のクラス (一部修正・再定義) ---

class AtomType(Enum):
    """原子タイプの定義 (分子や細胞成分の抽象化に利用)"""
    N = "N"      # 主鎖窒素 (例: タンパク質の一部)
    CA = "CA"    # α炭素
    C = "C"      # 主鎖炭素
    O = "O"      # 主鎖酸素
    CB = "CB"    # β炭素（側鎖）
    ION_NA = "Na+" # ナトリウムイオン
    ION_K = "K+"   # カリウムイオン
    ION_CA = "Ca2+"# カルシウムイオン
    DRUG_BINDING_SITE = "DrugBindingSite" # 薬物結合サイト
    MEMBRANE_LIPID = "MembraneLipid" # 細胞膜の一部
    ENZYME_ACTIVE_SITE = "EnzymeActiveSite" # 酵素活性部位
    # 新たな抽象的原子タイプ
    MYOCYTE_COMPONENT = "MyocyteComponent" # 心筋細胞の構成要素 (例: 収縮要素)
    VASCULAR_COMPONENT = "VascularComponent" # 血管の構成要素 (例: 血管壁細胞)
    ECM_COMPONENT = "ECMComponent" # 細胞外マトリックスの構成要素 (例: コラーゲン)

@dataclass
class MolecularCharacteristic:
    """
    分子の量子化学的特性を抽象的に表現。
    例: 親水性、電荷分布、特定の結合サイトの活性、反応性。
    これらは実際の量子化学計算の結果を「入力」として利用することを想定。
    """
    name: str
    hydrophobicity: float # -1.0 (親水性) から 1.0 (疎水性)
    charge_distribution: Dict[str, float] # AtomTypeごとの局所電荷傾向
    binding_affinity: Dict[str, float] # 他の特定のAtomTypeへの結合親和性 (例: "DRUG_BINDING_SITE": 0.8)
    reactivity: float # 反応性、例: 特定の酵素反応速度定数など
    size: float # 分子サイズ (例: nmスケール)

@dataclass
class Atom:
    """原子/分子/細胞構成要素の抽象的な表現"""
    atom_type: AtomType
    # residue_idxは、例えば「分子内のサブユニットID」や「細胞内のコンパートメントID」として再利用可能
    group_idx: int
    position: np.ndarray # 3D座標
    charge: float = 0.0 # 総電荷
    vdw_radius: float = 2.0 # ファンデルワールス半径 (相互作用距離の目安)
    # 量子化学的特性への参照 (任意)
    mol_char: Optional[MolecularCharacteristic] = None

    # 状態変数（時間変化する可能性のあるもの）
    concentration: float = 1.0 # 濃度 (例: イオン濃度)
    activity: float = 1.0 # 活性 (例: 酵素活性、チャネル開閉状態)
    health_status: float = 1.0 # 組織の健全性 (1.0 = 正常, 0.0 = 機能不全)

    # 相互作用状態
    bound_to: Optional[int] = None # 結合している他のAtomのgroup_idx

class OrientedSimplex:
    """向き付き単体 - 境界作用素の符号を正確に計算するため"""
    # (変更なし)
    def __init__(self, vertices: Tuple[int, ...]):
        self.vertices = tuple(sorted(vertices))
        self.dimension = len(vertices) - 1

    def boundary_with_signs(self) -> List[Tuple['OrientedSimplex', int]]:
        if self.dimension == 0:
            return []
        boundary = []
        for i, vertex in enumerate(self.vertices):
            face_vertices = self.vertices[:i] + self.vertices[i+1:]
            face = OrientedSimplex(face_vertices)
            sign = (-1) ** i
            boundary.append((face, sign))
        return boundary

class RigorousSimplicialComplex:
    """厳密な向き付き単体複体実装 (ネットワーク構造の表現に活用)"""
    # max_edge_lengthは「接続性」の閾値として再定義
    def __init__(self, atoms: List[Atom], max_edge_length: float = 10.0):
        self.atoms = atoms
        self.n_atoms = len(atoms)
        self.max_edge_length = max_edge_length # 接続性の最大距離
        self.positions = np.array([atom.position for atom in atoms])
        self._build_oriented_simplices()
        self._build_boundary_operators()

    def _build_oriented_simplices(self):
        """
        単体構築のロジックを、心臓モデルの「接続性」を表現するように調整。
        例: 細胞間の隣接、分子の結合、細胞内コンポーネントの配置。
        """
        distances = squareform(pdist(self.positions))

        self.vertices = [OrientedSimplex((i,)) for i in range(self.n_atoms)]
        self.edges = []
        for i in range(self.n_atoms):
            for j in range(i + 1, self.n_atoms):
                # ここで心臓モデルに特化したエッジの条件を追加
                if self._is_valid_heart_edge(self.atoms[i], self.atoms[j], distances[i, j]):
                    self.edges.append(OrientedSimplex((i, j)))

        self.triangles = []
        edge_set = {tuple(sorted(edge.vertices)) for edge in self.edges}
        for i in range(self.n_atoms):
            for j in range(i + 1, self.n_atoms):
                for k in range(j + 1, self.n_atoms):
                    if (self._all_edges_exist([(i,j), (i,k), (j,k)], edge_set) and
                        self._is_valid_heart_triangle(self.atoms[i], self.atoms[j], self.atoms[k])):
                        self.triangles.append(OrientedSimplex((i, j, k)))

        self.tetrahedra = []
        triangle_set = {tuple(sorted(tri.vertices)) for tri in self.triangles}
        for vertices in itertools.combinations(range(self.n_atoms), 4):
            if self._all_triangles_exist(vertices, triangle_set):
                self.tetrahedra.append(OrientedSimplex(vertices))

    def _is_valid_heart_edge(self, atom_i: Atom, atom_j: Atom, distance: float) -> bool:
        """
        心臓モデルにおける有効な「接続」の定義。
        例: 同じ細胞内の成分は常に結合、異なる細胞間の隣接は距離で判定。
        """
        # 細胞内コンポーネント間の結合
        if atom_i.group_idx == atom_j.group_idx and atom_i.group_idx >= 0: # group_idxが-1はグローバル分子
            return True # 同じ細胞/組織要素内のコンポーネントは結合しているとみなす

        # 細胞間/組織間の隣接
        if distance <= self.max_edge_length:
            # 特定の組み合わせで接続性を優先 (例: MYOCYTE_COMPONENT間の電気的結合)
            if {atom_i.atom_type, atom_j.atom_type} == {AtomType.MYOCYTE_COMPONENT, AtomType.MYOCYTE_COMPONENT}:
                return distance < 5.0 # 心筋細胞間の結合はより密接
            return True
        return False

    def _is_valid_heart_triangle(self, atom_i: Atom, atom_j: Atom, atom_k: Atom) -> bool:
        """心臓モデルにおける三角形の妥当性判定 (例: 空間的配置や機能的連結性)"""
        # 単純化のため、エッジが有効なら三角形も有効とする (より複雑なルールが可能)
        return True

    def _all_edges_exist(self, edge_list: List[Tuple[int, int]], edge_set: set) -> bool:
        return all(tuple(sorted(edge)) in edge_set for edge in edge_list)

    def _all_triangles_exist(self, vertices: Tuple[int, ...], triangle_set: set) -> bool:
        faces = list(itertools.combinations(vertices, 3))
        return all(tuple(sorted(face)) in triangle_set for face in faces)

    def _build_boundary_operators(self):
        self.boundary_1 = self._build_boundary_matrix(self.edges, self.vertices)
        self.boundary_2 = self._build_boundary_matrix(self.triangles, self.edges)
        self.boundary_3 = self._build_boundary_matrix(self.tetrahedra, self.triangles)

    def _build_boundary_matrix(self, high_dim_simplices: List[OrientedSimplex],
                                low_dim_simplices: List[OrientedSimplex]) -> sparse.csr_matrix:
        if not high_dim_simplices:
            return sparse.csr_matrix((len(low_dim_simplices), 0))
        low_dim_map = {tuple(sorted(simplex.vertices)): idx
                       for idx, simplex in enumerate(low_dim_simplices)}
        row_indices = []
        col_indices = []
        data = []
        for col_idx, simplex in enumerate(high_dim_simplices):
            boundary = simplex.boundary_with_signs()
            for face, sign in boundary:
                face_key = tuple(sorted(face.vertices))
                if face_key in low_dim_map:
                    row_idx = low_dim_map[face_key]
                    row_indices.append(row_idx)
                    col_indices.append(col_idx)
                    data.append(sign)
        return sparse.csr_matrix((data, (row_indices, col_indices)),
                                 shape=(len(low_dim_simplices), len(high_dim_simplices)))

    def compute_homology_groups(self) -> Dict[str, int]:
        def compute_betti_number(boundary_k, boundary_k_plus_1):
            if boundary_k_plus_1.shape[1] == 0:
                kernel_dim = boundary_k.shape[1] if boundary_k.shape[1] > 0 else 0
                image_dim = 0
            else:
                if boundary_k.shape[1] > 0:
                    kernel_dim = boundary_k.shape[1] - np.linalg.matrix_rank(boundary_k.toarray())
                else:
                    kernel_dim = 0
                image_dim = np.linalg.matrix_rank(boundary_k_plus_1.toarray())
            return max(0, kernel_dim - image_dim)

        betti_0 = self.n_atoms - np.linalg.matrix_rank(self.boundary_1.toarray()) if self.boundary_1.shape[1] > 0 else self.n_atoms
        betti_1 = compute_betti_number(self.boundary_1, self.boundary_2)
        betti_2 = compute_betti_number(self.boundary_2, self.boundary_3)

        return {
            'betti_0': betti_0,
            'betti_1': betti_1,
            'betti_2': betti_2,
            'euler_characteristic': betti_0 - betti_1 + betti_2
        }

class BiophysicalForceField:
    """
    生物物理学的力場 (心臓モデル向けに再解釈)。
    結合エネルギーは分子結合、角度エネルギーは分子の立体配置、
    VdW/静電は非共有結合性相互作用、H結合は特定の細胞シグナルなど、
    より抽象的なレベルでの相互作用に適用する。
    """

    def __init__(self):
        # パラメータは抽象化された「相互作用」を反映
        self.bond_params = self._init_bond_parameters()
        self.angle_params = self._init_angle_parameters()
        self.dihedral_params = self._init_dihedral_parameters()
        self.vdw_params = self._init_vdw_parameters() # 細胞膜の反発、分子の占有体積
        self.hbond_params = self._init_hbond_parameters() # 特定のシグナル伝達パス
        self.electrostatic_params = self._init_electrostatic_parameters() # イオン相互作用、膜電位

        self.energy_weights = {
            'bond': 100.0,      # 分子結合、細胞間接着
            'angle': 50.0,      # 分子構造、細胞内骨格の安定性
            'dihedral': 10.0,    # 構造変化、コンフォメーション
            'vdw': 5.0,          # 空間排除、細胞膜の相互作用
            'electrostatic': 10.0, # イオンチャネル、膜電位
            'hbond': 5.0         # 特異的分子認識、シグナル伝達
        }

    def _init_bond_parameters(self) -> Dict:
        """結合パラメータ (抽象化)"""
        return {
            frozenset({AtomType.ION_NA, AtomType.DRUG_BINDING_SITE}): {'r0': 3.0, 'k': 100.0}, # 薬物とNaチャネルの結合など
            frozenset({AtomType.MYOCYTE_COMPONENT, AtomType.MYOCYTE_COMPONENT}): {'r0': 4.0, 'k': 50.0}, # 心筋細胞間の接着結合
            frozenset({AtomType.ENZYME_ACTIVE_SITE, AtomType.DRUG_BINDING_SITE}): {'r0': 2.5, 'k': 200.0}, # 酵素-薬物相互作用
        }

    def _init_angle_parameters(self) -> Dict:
        """角度パラメータ (抽象化)"""
        return {
            # 例: 特定の細胞内構造の安定性
            (AtomType.MYOCYTE_COMPONENT, AtomType.MEMBRANE_LIPID, AtomType.MYOCYTE_COMPONENT): {'theta0': np.radians(109.5), 'k': 30.0},
        }

    def _init_dihedral_parameters(self) -> Dict:
        """二面角パラメータ (抽象化)"""
        return {
            # 例: イオンチャネルのコンフォメーション変化
            'channel_gate': {'V1': 0.1, 'V2': 0.5, 'V3': 0.1, 'gamma': 0.0},
        }

    def _init_vdw_parameters(self) -> Dict:
        """ファンデルワールスパラメータ (抽象化)"""
        return {
            AtomType.ION_NA: {'sigma': 2.0, 'epsilon': 0.05},
            AtomType.ION_K: {'sigma': 2.5, 'epsilon': 0.06},
            AtomType.ION_CA: {'sigma': 1.8, 'epsilon': 0.08},
            AtomType.DRUG_BINDING_SITE: {'sigma': 5.0, 'epsilon': 0.1},
            AtomType.MEMBRANE_LIPID: {'sigma': 8.0, 'epsilon': 0.2},
            AtomType.MYOCYTE_COMPONENT: {'sigma': 10.0, 'epsilon': 0.1},
            AtomType.ECM_COMPONENT: {'sigma': 15.0, 'epsilon': 0.15},
        }

    def _init_hbond_parameters(self) -> Dict:
        """水素結合パラメータ (抽象化: シグナル伝達の強さ)"""
        return {
            'ideal_distance': 3.0,
            'distance_cutoff': 5.0,
            'signal_strength': 10.0, # シグナル伝達による安定化/不安定化の度合い
        }

    def _init_electrostatic_parameters(self) -> Dict:
        """静電相互作用パラメータ"""
        return {
            'dielectric_constant_intra_cell': 4.0, # 細胞内
            'dielectric_constant_inter_cell': 80.0, # 細胞間（水環境）
            'coulomb_constant': 332.0, # kcal*Å/mol/e^2
        }

    def compute_total_energy(self, atoms: List[Atom]) -> float:
        energy = 0.0
        energy += self.energy_weights['bond'] * self._bond_energy(atoms)
        energy += self.energy_weights['angle'] * self._angle_energy(atoms)
        energy += self.energy_weights['dihedral'] * self._dihedral_energy(atoms)
        energy += self.energy_weights['vdw'] * self._vdw_energy(atoms)
        energy += self.energy_weights['electrostatic'] * self._electrostatic_energy(atoms)
        energy += self.energy_weights['hbond'] * self._hbond_energy(atoms) # シグナル伝達として

        # 細胞の健康状態に基づく追加ペナルティ/ボーナス
        for atom in atoms:
            if atom.atom_type in [AtomType.MYOCYTE_COMPONENT, AtomType.VASCULAR_COMPONENT]:
                energy += (1.0 - atom.health_status) * 100.0 # 健康状態が悪いほど高エネルギー（不安定）

        return energy

    def _bond_energy(self, atoms: List[Atom]) -> float:
        """抽象化された結合エネルギー (例: 細胞接着、分子結合)"""
        energy = 0.0
        for i, atom_i in enumerate(atoms):
            for j, atom_j in enumerate(atoms[i+1:], i+1):
                bond_key = frozenset({atom_i.atom_type, atom_j.atom_type})
                if bond_key in self.bond_params and self._is_abstract_bonded(atom_i, atom_j):
                    r = np.linalg.norm(atom_j.position - atom_i.position)
                    r0 = self.bond_params[bond_key]['r0']
                    k = self.bond_params[bond_key]['k']
                    energy += k * (r - r0)**2
        return energy

    def _angle_energy(self, atoms: List[Atom]) -> float:
        """抽象化された角度エネルギー (例: 細胞骨格の安定性)"""
        energy = 0.0
        # これは非常に抽象的であり、特定の3原子結合に基づいて計算する
        # 例えば、特定の細胞内コンポーネントが特定の角度を保つ傾向がある場合など
        return energy

    def _dihedral_energy(self, atoms: List[Atom]) -> float:
        """抽象化された二面角エネルギー (例: 分子のコンフォメーション変化、チャネル開閉)"""
        energy = 0.0
        # 例: イオンチャネルゲートの開閉を模倣
        # 4つの原子がチャネルのゲートを形成していると仮定
        # これは簡略化のために、特定のチャネル原子タイプの存在で適用
        channel_gate_atoms = [a for a in atoms if a.atom_type == AtomType.DRUG_BINDING_SITE] # 仮にDRUG_BINDING_SITEがチャネルの一部
        if len(channel_gate_atoms) >= 4:
            # 簡略化のため、最初の4つのチャネルゲート原子で二面角を計算
            pos1, pos2, pos3, pos4 = [a.position for a in channel_gate_atoms[:4]]
            dihedral = self._calculate_dihedral(pos1, pos2, pos3, pos4)
            energy += self._dihedral_potential(dihedral, 'channel_gate')
        return energy

    def _vdw_energy(self, atoms: List[Atom]) -> float:
        """ファンデルワールスエネルギー (空間排除、非特異的相互作用)"""
        energy = 0.0
        for i, atom_i in enumerate(atoms):
            for j, atom_j in enumerate(atoms[i+1:], i+1):
                # 結合している原子間のVdWは考慮しない
                if not self._is_abstract_bonded(atom_i, atom_j):
                    r = np.linalg.norm(atom_j.position - atom_i.position)
                    if atom_i.atom_type not in self.vdw_params or atom_j.atom_type not in self.vdw_params:
                        continue
                    sigma_i = self.vdw_params[atom_i.atom_type]['sigma']
                    sigma_j = self.vdw_params[atom_j.atom_type]['sigma']
                    epsilon_i = self.vdw_params[atom_i.atom_type]['epsilon']
                    epsilon_j = self.vdw_params[atom_j.atom_type]['epsilon']
                    sigma = (sigma_i + sigma_j) / 2
                    epsilon = np.sqrt(epsilon_i * epsilon_j)
                    if r < 0.1: r = 0.1
                    energy += 4 * epsilon * ((sigma/r)**12 - (sigma/r)**6)
        return energy

    def _hbond_energy(self, atoms: List[Atom]) -> float:
        """抽象化された水素結合エネルギー (特定のシグナル伝達)"""
        energy = 0.0
        # 例えば、受容体とリガンドの相互作用を模倣
        # DRUG_BINDING_SITEとION_NAが特定のシグナル伝達を持つと仮定
        for atom_d in atoms: # Donor-like
            if atom_d.atom_type == AtomType.DRUG_BINDING_SITE:
                for atom_a in atoms: # Acceptor-like
                    if atom_a.atom_type == AtomType.ION_NA and atom_d.group_idx != atom_a.group_idx: # 異なるグループ間
                        distance = np.linalg.norm(atom_a.position - atom_d.position)
                        if distance <= self.hbond_params['distance_cutoff']:
                            ideal_dist = self.hbond_params['ideal_distance']
                            # 結合が起こるとエネルギーが下がる (安定化)
                            energy -= self.hbond_params['signal_strength'] * np.exp(-(distance - ideal_dist)**2 / 1.0)
        return energy

    def _electrostatic_energy(self, atoms: List[Atom]) -> float:
        """静電相互作用エネルギー (イオンチャネル、膜電位)"""
        energy = 0.0
        coulomb_constant = self.electrostatic_params['coulomb_constant']

        for i, atom_i in enumerate(atoms):
            for j, atom_j in enumerate(atoms[i+1:], i+1):
                if not self._is_abstract_bonded(atom_i, atom_j):
                    r = np.linalg.norm(atom_j.position - atom_i.position)
                    if r > 0.1:
                        # 異なるグループ間であれば細胞外の誘電率を、同じグループであれば細胞内の誘電率を使用
                        dielectric = (self.electrostatic_params['dielectric_constant_inter_cell']
                                      if atom_i.group_idx != atom_j.group_idx else
                                      self.electrostatic_params['dielectric_constant_intra_cell'])
                        energy += coulomb_constant * atom_i.charge * atom_j.charge / (dielectric * r)
        return energy

    # ヘルパーメソッド (心臓モデル向けに修正)
    def _is_abstract_bonded(self, atom1: Atom, atom2: Atom) -> bool:
        """抽象化された結合判定 (例: 共有結合、機能的結合)"""
        # 同じグループ内のものは結合とみなす
        if atom1.group_idx == atom2.group_idx and atom1.group_idx >= 0:
            return True

        # 特定の分子タイプ間の結合
        bond_pairs = [
            frozenset({AtomType.ION_NA, AtomType.DRUG_BINDING_SITE}),
            frozenset({AtomType.MYOCYTE_COMPONENT, AtomType.MYOCYTE_COMPONENT})
        ]
        atom_pair = frozenset({atom1.atom_type, atom2.atom_type})

        # 空間的近接性に基づく結合 (例: 細胞間の接着)
        if atom_pair in bond_pairs:
            return True # 詳細な距離判定は力場任せ

        return False

    def _calculate_angle(self, pos1: np.ndarray, pos2: np.ndarray, pos3: np.ndarray) -> float:
        v1 = pos1 - pos2
        v2 = pos3 - pos2
        cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-8)
        return np.arccos(np.clip(cos_angle, -1, 1))

    def _calculate_dihedral(self, pos1: np.ndarray, pos2: np.ndarray,
                            pos3: np.ndarray, pos4: np.ndarray) -> float:
        b1 = pos2 - pos1
        b2 = pos3 - pos2
        b3 = pos4 - pos3
        n1 = np.cross(b1, b2)
        n2 = np.cross(b2, b3)
        n1 = n1 / (np.linalg.norm(n1) + 1e-8)
        n2 = n2 / (np.linalg.norm(n2) + 1e-8)
        cos_dihedral = np.dot(n1, n2)
        sin_dihedral = np.dot(np.cross(n1, n2), b2 / (np.linalg.norm(b2) + 1e-8))
        return np.arctan2(sin_dihedral, cos_dihedral)

    def _dihedral_potential(self, angle: float, angle_type: str) -> float:
        params = self.dihedral_params[angle_type]
        energy = 0.0
        if 'V1' in params: energy += params['V1'] * (1 + np.cos(angle + params['gamma']))
        if 'V2' in params: energy += params['V2'] * (1 - np.cos(2*angle + params['gamma']))
        if 'V3' in params: energy += params['V3'] * (1 + np.cos(3*angle + params['gamma']))
        return energy


# --- 新規クラス群 (心臓症例モデル向け) ---

@dataclass
class TissueElement:
    """
    心臓組織の構成要素を表現する抽象クラス。
    例: 心筋細胞、血管、細胞外マトリックスなど。
    """
    element_id: int
    element_type: str # 'Myocyte', 'Vessel', 'ECM'
    component_atoms: List[Atom] = field(default_factory=list) # 内部に含むAtomのリスト
    health_status: float = 1.0 # 組織要素全体の健康状態 (0.0: 機能不全, 1.0: 正常)
    current_state: Dict[str, float] = field(default_factory=dict) # 膜電位、収縮力など

    def update_health(self, change: float):
        """健康状態を更新し、内部のAtomにも伝播"""
        self.health_status = np.clip(self.health_status + change, 0.0, 1.0)
        for atom in self.component_atoms:
            atom.health_status = self.health_status # 内部アトムの健康状態も更新

    def get_center_position(self) -> np.ndarray:
        """この組織要素の中心座標を計算"""
        if not self.component_atoms:
            return np.array([0.0, 0.0, 0.0])
        positions = np.array([atom.position for atom in self.component_atoms])
        return np.mean(positions, axis=0)

class CausalEngine:
    """
    心臓症例の因果推論ロジックを管理するエンジン。
    特定の介入がシステム全体にどのように影響を及ぼすかをシミュレートする。
    """

    def __init__(self, force_field: BiophysicalForceField):
        self.force_field = force_field
        self.molecular_characteristics: Dict[str, MolecularCharacteristic] = self._load_molecular_characteristics()

    def _load_molecular_characteristics(self) -> Dict[str, MolecularCharacteristic]:
        """
        量子化学計算から得られるような分子の特性データをロードする。
        これはシミュレーションの「入力データ」となる。
        """
        return {
            "DrugA": MolecularCharacteristic(
                name="DrugA",
                hydrophobicity=0.7,
                charge_distribution={"DRUG_BINDING_SITE": -0.5, "C": 0.1},
                binding_affinity={"ION_NA": 0.9, "ENZYME_ACTIVE_SITE": 0.2}, # Naチャネルへの親和性が高い
                reactivity=0.1,
                size=3.0
            ),
            "DrugB": MolecularCharacteristic(
                name="DrugB",
                hydrophobicity=-0.3,
                charge_distribution={"DRUG_BINDING_SITE": 0.2, "N": -0.3},
                binding_affinity={"ION_K": 0.8, "DRUG_BINDING_SITE": 0.1}, # Kチャネルへの親和性が高い
                reactivity=0.05,
                size=2.5
            ),
            "Na_Ion": MolecularCharacteristic(
                name="Na_Ion", hydrophobicity=-0.9, charge_distribution={"ION_NA": 1.0}, binding_affinity={}, reactivity=0.0, size=0.2
            ),
            "K_Ion": MolecularCharacteristic(
                name="K_Ion", hydrophobicity=-0.9, charge_distribution={"ION_K": 1.0}, binding_affinity={}, reactivity=0.0, size=0.2
            ),
            "Myosin": MolecularCharacteristic( # 収縮タンパク質を模倣
                name="Myosin", hydrophobicity=0.1, charge_distribution={"MYOCYTE_COMPONENT": -0.2}, binding_affinity={}, reactivity=0.0, size=10.0
            ),
            "Actin": MolecularCharacteristic( # 収縮タンパク質を模倣
                name="Actin", hydrophobicity=0.0, charge_distribution={"MYOCYTE_COMPONENT": 0.1}, binding_affinity={}, reactivity=0.0, size=8.0
            ),
            # ... 他の分子特性
        }

    def apply_intervention(self, atoms: List[Atom], tissue_elements: List[TissueElement],
                           intervention_type: str, params: Dict) -> Tuple[List[Atom], List[TissueElement]]:
        """
        特定の介入をシステムに適用し、その初期影響を計算する。
        例: 薬物投与、遺伝子変異、環境ストレス。
        """
        print(f"\n--- Applying Intervention: {intervention_type} ---")
        if intervention_type == "DrugAdministration":
            drug_name = params.get("drug_name", "DrugA")
            drug_concentration = params.get("concentration", 0.1) # nM or uM, etc.

            mol_char = self.molecular_characteristics.get(drug_name)
            if not mol_char:
                print(f"Warning: Drug {drug_name} not found in molecular characteristics.")
                return atoms, tissue_elements

            # 薬物が特定の結合サイトに到達し、影響を与えることをシミュレート
            # 例: Naチャネルの結合サイトに薬物が結合
            for atom in atoms:
                if atom.atom_type == AtomType.DRUG_BINDING_SITE:
                    # 薬物の結合親和性と現在の濃度に応じて結合確率を決定
                    binding_prob = mol_char.binding_affinity.get(atom.atom_type.value, 0.0) * drug_concentration
                    if random.random() < binding_prob:
                        atom.bound_to = -1 # 薬物（グローバル分子なのでIDは-1など）に結合したとマーク
                        atom.activity *= (1 - mol_char.reactivity) # 薬物結合による活性変化

                        # 結合サイトが存在する細胞/組織要素の健康状態に影響
                        for tissue in tissue_elements:
                            if any(a.atom_type == atom.atom_type and a.group_idx == atom.group_idx for a in tissue.component_atoms):
                                # 薬物特性 (例: 毒性、治療効果) に基づいて健康状態を更新
                                health_change = -0.05 if mol_char.reactivity > 0.5 else 0.03 # 毒性が高いと減少、低いと増加
                                tissue.update_health(health_change)
                                print(f"  {drug_name} bound to {atom.atom_type} in tissue {tissue.element_id}. Activity changed to {atom.activity:.2f}.")
                                print(f"  Tissue {tissue.element_id} health: {tissue.health_status:.2f}")

        elif intervention_type == "GeneticMutation":
            target_type = params.get("target_type", AtomType.ION_NA)
            mutation_effect = params.get("effect", 0.5) # 0.5 means 50% reduction in activity

            print(f"  Applying genetic mutation effect ({mutation_effect*100:.0f}% reduction) to {target_type} atoms.")
            for atom in atoms:
                if atom.atom_type == target_type:
                    atom.activity *= (1 - mutation_effect)
                    # 関連する組織の健康状態にも影響
                    for tissue in tissue_elements:
                        if any(a.atom_type == atom.atom_type and a.group_idx == atom.group_idx for a in tissue.component_atoms):
                            tissue.update_health(-0.1 * mutation_effect) # 突然変異は健康を悪化させる傾向
                            print(f"  Atom {atom.atom_type} in tissue {tissue.element_id} activity changed to {atom.activity:.2f}. Health: {tissue.health_status:.2f}")

        # その他の介入 (例: 虚血、炎症など) も追加可能
        return atoms, tissue_elements

    def propagate_causal_effects(self, atoms: List[Atom], tissue_elements: List[TissueElement],
                                 simplicial_complex: RigorousSimplicialComplex) -> Tuple[List[Atom], List[TissueElement]]:
        """
        介入による影響をシステム全体に伝播させる。
        これは、分子レベルから組織レベルへの橋渡しを行う主要なステップ。
        シンプリシャル複体のトポロジー情報も利用する。
        """
        print("\n--- Propagating Causal Effects ---")

        # 1. 分子レベルの影響伝播 (例: イオン濃度の変化、酵素反応)
        # イオンチャネルの活性とイオン濃度
        for atom in atoms:
            if atom.atom_type == AtomType.ION_NA:
                # Naチャネルの活性 (DRUG_BINDING_SITEの活性から影響を受けると仮定)
                # 同じグループ内のDRUG_BINDING_SITEの平均活性からNaイオン濃度を決定
                if atom.group_idx >= 0:
                    related_site_atoms = [a for a in atoms if a.group_idx == atom.group_idx and a.atom_type == AtomType.DRUG_BINDING_SITE]
                    if related_site_atoms:
                        avg_site_activity = np.mean([a.activity for a in related_site_atoms])
                        # チャネル活性が高いと細胞内Na濃度が増加すると仮定
                        atom.concentration = np.clip(atom.concentration * (1 + (1 - avg_site_activity) * 0.1), 0.1, 2.0) # 簡略化
                        # 濃度変化は健康状態にも影響
                        for tissue in tissue_elements:
                            if tissue.element_id == atom.group_idx:
                                if atom.concentration > 1.5: tissue.update_health(-0.02) # 高Naは細胞毒性
                                elif atom.concentration < 0.5: tissue.update_health(-0.01) # 低Naも問題
                                print(f"  Ion {atom.atom_type} in tissue {tissue.element_id} concentration: {atom.concentration:.2f}. Health: {tissue.health_status:.2f}")

        # 2. 組織レベルへの影響伝播 (例: 細胞間のシグナル伝達、力学的相互作用)
        # シンプリシャル複体のエッジ（結合）を利用して隣接する組織要素に影響を伝播
        adjacency_matrix = simplicial_complex.boundary_1 @ simplicial_complex.boundary_1.T
        adj_graph = nx.from_scipy_sparse_array(adjacency_matrix)

        for i, current_atom in enumerate(atoms):
            if current_atom.atom_type in [AtomType.MYOCYTE_COMPONENT, AtomType.VASCULAR_COMPONENT]:
                current_tissue = next((t for t in tissue_elements if t.element_id == current_atom.group_idx), None)
                if not current_tissue: continue

                # 隣接する組織要素に健康状態を伝播 (簡易拡散モデル)
                neighbors_indices = list(adj_graph.neighbors(i))
                neighbor_tissues = []
                for neighbor_atom_idx in neighbors_indices:
                    neighbor_atom = atoms[neighbor_atom_idx]
                    neighbor_tissue = next((t for t in tissue_elements if t.element_id == neighbor_atom.group_idx), None)
                    if neighbor_tissue and neighbor_tissue.element_id != current_tissue.element_id: # 異なる組織要素の場合
                        neighbor_tissues.append(neighbor_tissue)

                for n_tissue in neighbor_tissues:
                    # 健康状態の差分に基づいて伝播
                    diff_health = current_tissue.health_status - n_tissue.health_status
                    if abs(diff_health) > 0.05: # ある程度の差がある場合のみ伝播
                        change = diff_health * 0.01 # 伝播係数
                        n_tissue.update_health(change)
                        print(f"  Tissue {current_tissue.element_id} propagated health to {n_tissue.element_id}. Health: {n_tissue.health_status:.2f}")

        # 3. 組織要素内部の状態更新 (例: 収縮力、膜電位)
        for tissue in tissue_elements:
            if tissue.element_type == 'Myocyte':
                # イオン濃度に基づいて収縮力を計算 (非常に簡略化)
                na_ion_atoms = [a for a in tissue.component_atoms if a.atom_type == AtomType.ION_NA]
                ca_ion_atoms = [a for a in tissue.component_atoms if a.atom_type == AtomType.ION_CA] # カルシウムイオンも重要

                avg_na_conc = np.mean([a.concentration for a in na_ion_atoms]) if na_ion_atoms else 1.0
                # ここでCaイオンの流入や放出もモデル化されるべきだが、今回はNa濃度で代用

                # 収縮力はNa濃度と健康状態に依存すると仮定
                contraction_force = np.clip(tissue.health_status * (1.0 + (1.0 - avg_na_conc) * 0.5), 0.0, 1.0)
                tissue.current_state['contraction_force'] = contraction_force
                print(f"  Myocyte {tissue.element_id} contraction force: {contraction_force:.2f}")

            # その他の組織要素の状態更新ロジック

        return atoms, tissue_elements

    def assess_case_outcome(self, tissue_elements: List[TissueElement]) -> Dict:
        """
        現在のシステム状態に基づいて、症例のアウトカムを評価する。
        """
        print("\n--- Assessing Case Outcome ---")
        overall_health = np.mean([t.health_status for t in tissue_elements])

        # 収縮力の平均
        myocyte_forces = [t.current_state.get('contraction_force', 0.0) for t in tissue_elements if t.element_type == 'Myocyte']
        avg_contraction_force = np.mean(myocyte_forces) if myocyte_forces else 0.0

        outcome_summary = {
            "overall_tissue_health": overall_health,
            "average_myocyte_contraction_force": avg_contraction_force,
            "potential_arrhythmia_risk": 1.0 - avg_contraction_force, # 収縮力低下はリスク上昇と仮定
            "prognosis": "Stable" if overall_health > 0.8 and avg_contraction_force > 0.7 else "Compromised"
        }
        print(f"  Overall Tissue Health: {overall_health:.2f}")
        print(f"  Average Myocyte Contraction Force: {avg_contraction_force:.2f}")
        print(f"  Prognosis: {outcome_summary['prognosis']}")
        return outcome_summary

class CardiacModelSimulator:
    """
    心臓症例モデルのシミュレーション全体を管理するメインクラス。
    """

    def __init__(self):
        self.force_field = BiophysicalForceField()
        self.causal_engine = CausalEngine(self.force_field)
        self.atoms: List[Atom] = []
        self.tissue_elements: List[TissueElement] = []

    def build_initial_heart_model(self, num_myocytes: int = 5, num_vessels: int = 2) -> Tuple[List[Atom], List[TissueElement]]:
        """
        心臓の抽象的な初期モデルを構築する。
        複数の心筋細胞と血管から構成されると仮定。
        """
        print("Building initial abstract heart model...")
        atoms: List[Atom] = []
        tissue_elements: List[TissueElement] = []
        atom_id_counter = 0

        # 心筋細胞 (Myocytes)
        for i in range(num_myocytes):
            myocyte_id = i
            myocyte_atoms: List[Atom] = []

            # 中心座標をランダムに配置
            center_pos = np.array([random.uniform(-50, 50), random.uniform(-50, 50), random.uniform(-50, 50)])

            # Myocyte Component (収縮要素など)
            myocyte_atoms.append(Atom(AtomType.MYOCYTE_COMPONENT, myocyte_id, center_pos + np.array([0,0,0]),
                                     charge=0.0, vdw_radius=10.0))
            # イオンチャネル (薬物結合サイトを含む)
            myocyte_atoms.append(Atom(AtomType.DRUG_BINDING_SITE, myocyte_id, center_pos + np.array([5,5,5]),
                                     charge=0.0, vdw_radius=3.0)) # Naチャネルの一部を模倣
            # イオン
            myocyte_atoms.append(Atom(AtomType.ION_NA, myocyte_id, center_pos + np.array([1,1,1]), charge=1.0,
                                     vdw_radius=0.5, concentration=1.0,
                                     mol_char=self.causal_engine.molecular_characteristics["Na_Ion"]))
            myocyte_atoms.append(Atom(AtomType.ION_K, myocyte_id, center_pos + np.array([-1,-1,-1]), charge=1.0,
                                     vdw_radius=0.5, concentration=1.0,
                                     mol_char=self.causal_engine.molecular_characteristics["K_Ion"]))
            myocyte_atoms.append(Atom(AtomType.ION_CA, myocyte_id, center_pos + np.array([2,-2,0]), charge=2.0,
                                     vdw_radius=0.5, concentration=0.1)) # 細胞内Ca濃度は低い

            # 他の細胞内分子 (例: Myosin, Actin)
            myocyte_atoms.append(Atom(AtomType.CA, myocyte_id, center_pos + np.array([8,0,0]), charge=0.0,
                                     vdw_radius=5.0, mol_char=self.causal_engine.molecular_characteristics["Myosin"]))
            myocyte_atoms.append(Atom(AtomType.C, myocyte_id, center_pos + np.array([-8,0,0]), charge=0.0,
                                     vdw_radius=5.0, mol_char=self.causal_engine.molecular_characteristics["Actin"]))


            # 各AtomにユニークなIDを割り当て
            for atom in myocyte_atoms:
                atom.global_id = atom_id_counter
                atoms.append(atom)
                atom_id_counter += 1

            tissue_elements.append(TissueElement(myocyte_id, 'Myocyte', myocyte_atoms))

        # 血管 (Vessels)
        for i in range(num_vessels):
            vessel_id = num_myocytes + i
            vessel_atoms: List[Atom] = []

            center_pos = np.array([random.uniform(-50, 50), random.uniform(-50, 50), random.uniform(-50, 50)])

            vessel_atoms.append(Atom(AtomType.VASCULAR_COMPONENT, vessel_id, center_pos,
                                     charge=0.0, vdw_radius=12.0))

            for atom in vessel_atoms:
                atom.global_id = atom_id_counter
                atoms.append(atom)
                atom_id_counter += 1

            tissue_elements.append(TissueElement(vessel_id, 'Vessel', vessel_atoms))

        self.atoms = atoms
        self.tissue_elements = tissue_elements
        print(f"  Model built with {len(atoms)} atoms and {len(tissue_elements)} tissue elements.")
        return atoms, tissue_elements

    def _coordinates_to_atoms(self, coords: np.ndarray, initial_atoms: List[Atom]) -> List[Atom]:
        """最適化された座標から原子リストを再構築"""
        updated_atoms = []
        for i, atom in enumerate(initial_atoms):
            updated_atoms.append(Atom(atom.atom_type, atom.group_idx, coords[i],
                                      atom.charge, atom.vdw_radius, atom.mol_char,
                                      atom.concentration, atom.activity, atom.health_status, atom.bound_to))
        return updated_atoms

    def simulate_case(self, intervention_type: str, intervention_params: Dict,
                      simulation_steps: int = 5, optimization_iterations: int = 500) -> Dict:
        """
        症例シミュレーションの実行。
        初期モデル構築 -> 介入 -> エネルギー最小化 -> 因果伝播 -> アウトカム評価
        """
        print("\n--- Starting Cardiac Case Simulation ---")

        # 1. 初期モデルの構築
        initial_atoms, initial_tissue_elements = self.build_initial_heart_model()
        current_atoms = initial_atoms
        current_tissue_elements = initial_tissue_elements

        # 2. 介入の適用
        current_atoms, current_tissue_elements = self.causal_engine.apply_intervention(
            current_atoms, current_tissue_elements, intervention_type, intervention_params
        )

        # シミュレーションループ
        for step in range(simulation_steps):
            print(f"\n--- Simulation Step {step + 1}/{simulation_steps} ---")

            # 3. 構造の最適化 (エネルギー最小化)
            # 現在の原子位置を最適化対象とする
            initial_coords_for_opt = np.array([atom.position for atom in current_atoms])

            def objective_function(coords_flat: np.ndarray) -> float:
                coords = coords_flat.reshape(-1, 3)
                temp_atoms = self._coordinates_to_atoms(coords, current_atoms) # temporary atoms for energy calc

                # 生物物理学的エネルギー計算
                energy = self.force_field.compute_total_energy(temp_atoms)

                # トポロジー的ペナルティ (オプション)
                # 例えば、組織が物理的に断片化したり、不自然な連結が生じたりした場合にペナルティ
                try:
                    sim_complex = RigorousSimplicialComplex(temp_atoms)
                    homology = sim_complex.compute_homology_groups()
                    # 健康状態が悪い組織は、ホモロジーも悪化しやすいと仮定
                    avg_health_in_step = np.mean([t.health_status for t in current_tissue_elements])

                    if homology['betti_0'] > len(current_tissue_elements) * 1.2: # 断片化が過度に進んだ場合
                        energy += 1000.0 * (homology['betti_0'] - len(current_tissue_elements)) * (1 - avg_health_in_step)
                    if homology['betti_1'] > len(current_atoms) // 5: # 不自然なループ
                         energy += 50.0 * (homology['betti_1'] - len(current_atoms) // 5) * (1 - avg_health_in_step)

                except Exception as e:
                    # warnings.warn(f"Simplicial complex calculation failed: {e}. Large penalty applied.")
                    energy += 1e7 # 大幅な構造の破綻

                if not np.isfinite(energy):
                    return 1e10
                return energy

            print(f"  Optimizing structure for step {step+1}...")
            # L-BFGS-B は勾配計算が必要だが、ここでは自動微分は行わないので、純粋な数値微分に近い
            result = minimize(
                objective_function,
                initial_coords_for_opt.flatten(),
                method='L-BFGS-B',
                options={'maxiter': optimization_iterations, 'ftol': 1e-4, 'disp': False} # disp=Falseで冗長な出力を抑制
            )

            optimized_coords = result.x.reshape(-1, 3)
            current_atoms = self._coordinates_to_atoms(optimized_coords, current_atoms)
            print(f"  Optimization completed. Energy: {result.fun:.2f}")

            # 各原子のhealth_statusをその所属する組織要素のhealth_statusと同期させる
            # これは、組織レベルの変化が分子レベルにも影響を与えるという因果パスを表現する
            for atom in current_atoms:
                if atom.group_idx >= 0:
                    parent_tissue = next((t for t in current_tissue_elements if t.element_id == atom.group_idx), None)
                    if parent_tissue:
                        atom.health_status = parent_tissue.health_status

            # 4. 因果影響の伝播
            current_simplicial_complex = RigorousSimplicialComplex(current_atoms)
            current_atoms, current_tissue_elements = self.causal_engine.propagate_causal_effects(
                current_atoms, current_tissue_elements, current_simplicial_complex
            )

            # 各TissueElementのcomponent_atomsのリストも最新の原子リストから再構築する必要がある
            # 最適化によって原子の順序が変わることはないが、状態は更新されているため、参照を更新
            for tissue in current_tissue_elements:
                tissue.component_atoms = [a for a in current_atoms if a.group_idx == tissue.element_id]

        # 5. 最終的な症例アウトカムの評価
        final_outcome = self.causal_engine.assess_case_outcome(current_tissue_elements)

        print("\n--- Cardiac Case Simulation Complete ---")
        return final_outcome

    def visualize_heart_model(self, atoms: List[Atom], tissue_elements: List[TissueElement], title: str = "Heart Model"):
        """
        心臓モデルを可視化する (組織要素と内部コンポーネント)。
        健康状態やタイプに応じて色を変える。
        """
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')

        colors = {'Myocyte': 'red', 'Vessel': 'blue', 'ECM': 'green'}
        atom_colors = {
            AtomType.MYOCYTE_COMPONENT: 'red',
            AtomType.DRUG_BINDING_SITE: 'purple',
            AtomType.ION_NA: 'orange',
            AtomType.ION_K: 'gold',
            AtomType.ION_CA: 'lime',
            AtomType.VASCULAR_COMPONENT: 'blue',
            AtomType.MEMBRANE_LIPID: 'cyan',
            AtomType.ENZYME_ACTIVE_SITE: 'magenta',
            AtomType.CA: 'gray', # Myosin/Actin proxy
            AtomType.C: 'darkgray',
            AtomType.N: 'pink',
            AtomType.O: 'brown',
            AtomType.CB: 'olive'
        }

        # 組織要素ごとにプロット
        for tissue in tissue_elements:
            tissue_center = tissue.get_center_position()
            tissue_color = colors.get(tissue.element_type, 'gray')

            # 組織要素の中心を大きなマーカーで表示し、健康状態に応じて透明度を変える
            ax.scatter(tissue_center[0], tissue_center[1], tissue_center[2],
                       s=tissue.health_status * 500 + 100,
                       color=tissue_color, alpha=tissue.health_status,
                       label=f'{tissue.element_type} (ID:{tissue.element_id}, Health:{tissue.health_status:.2f})' if tissue.element_id == 0 else "") # ラベルは一つだけ表示

            # 内部の原子をプロット
            for atom in tissue.component_atoms:
                ax.scatter(atom.position[0], atom.position[1], atom.position[2],
                           s=20 + atom.vdw_radius*2,
                           color=atom_colors.get(atom.atom_type, 'black'),
                           alpha=atom.activity) # 活性に応じて透明度を変える

        # 接続性（エッジ）を描画（Simplicial Complexから）
        sim_complex = RigorousSimplicialComplex(atoms)
        for edge in sim_complex.edges:
            p1 = atoms[edge.vertices[0]].position
            p2 = atoms[edge.vertices[1]].position
            ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]],
                    'k-', linewidth=0.5, alpha=0.3) # 黒線、薄く

        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(title)

        # 凡例の調整
        handles, labels = ax.get_legend_handles_labels()
        unique_labels = list(set(labels))
        unique_handles = [handles[labels.index(ul)] for ul in unique_labels]
        ax.legend(unique_handles, unique_labels, loc='upper left', bbox_to_anchor=(1.05, 1))

        plt.tight_layout(rect=[0, 0, 0.85, 1]) # 凡例のためにスペースを確保
        plt.show()


# --- メイン実行ブロック ---
if __name__ == "__main__":
    simulator = CardiacModelSimulator()

    # シミュレーションの実行例1: 薬物投与 (Naチャネル阻害薬を模倣)
    print("--- Scenario 1: Simulating Drug Administration (e.g., Na+ channel blocker) ---")
    outcome1 = simulator.simulate_case(
        intervention_type="DrugAdministration",
        intervention_params={"drug_name": "DrugA", "concentration": 0.5},
        simulation_steps=3, # 短いステップ数でデモンストレーション
        optimization_iterations=2 # 短い反復回数
    )
    print("\nFinal Outcome for Drug Administration Scenario:")
    print(outcome1)
    # 最終的なモデル状態を可視化
    simulator.visualize_heart_model(simulator.atoms, simulator.tissue_elements,
                                    title="Cardiac Model After Drug Administration")

    print("\n" + "="*80 + "\n")

    # シミュレーションの実行例2: 遺伝子変異 (K+チャネル機能不全を模倣)
    print("--- Scenario 2: Simulating Genetic Mutation (e.g., K+ channel dysfunction) ---")
    # シミュレータをリセットして初期モデルを再構築
    simulator = CardiacModelSimulator()
    outcome2 = simulator.simulate_case(
        intervention_type="GeneticMutation",
        intervention_params={"target_type": AtomType.ION_K, "effect": 0.7}, # K+チャネル活性を70%低下
        simulation_steps=3,
        optimization_iterations=2
    )
    print("\nFinal Outcome for Genetic Mutation Scenario:")
    print(outcome2)
    simulator.visualize_heart_model(simulator.atoms, simulator.tissue_elements,
                                    title="Cardiac Model After Genetic Mutation")

    print("\nSimulation complete for all scenarios.")

--- Scenario 1: Simulating Drug Administration (e.g., Na+ channel blocker) ---

--- Starting Cardiac Case Simulation ---
Building initial abstract heart model...
  Model built with 37 atoms and 7 tissue elements.

--- Applying Intervention: DrugAdministration ---

--- Simulation Step 1/3 ---
  Optimizing structure for step 1...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-2-1527840816.py", line 912, in <cell line: 0>
    outcome1 = simulator.simulate_case(
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-2-1527840816.py", line 804, in simulate_case
    result = minimize(
             ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/scipy/optimize/_minimize.py", line 738, in minimize
    res = _minimize_lbfgsb(fun, x0, args, jac, bounds,
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/scipy/optimize/_lbfgsb_py.py", line 386, in _minimize_lbfgsb
    sf = _prepare_scalar_function(fun, x0, jac=jac, args=args, epsilon=eps,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/scipy/optimize/

NameError: name 'desired_reduced_dimension' is not defined